In [ ]:
import torch
import numpy as np
import random
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
import torch
import torchvision.ops
from torch import nn

class DeformableConv2d(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=1,
                 bias=False):

        super(DeformableConv2d, self).__init__()

        self.padding = padding
        
        self.offset_conv = nn.Conv2d(in_channels, 
                                     2 * kernel_size * kernel_size,
                                     kernel_size=kernel_size, 
                                     stride=stride,
                                     padding=self.padding, 
                                     bias=True)

        nn.init.constant_(self.offset_conv.weight, 0.)
        nn.init.constant_(self.offset_conv.bias, 0.)
        
        self.modulator_conv = nn.Conv2d(in_channels, 
                                     1 * kernel_size * kernel_size,
                                     kernel_size=kernel_size, 
                                     stride=stride,
                                     padding=self.padding, 
                                     bias=True)

        nn.init.constant_(self.modulator_conv.weight, 0.)
        nn.init.constant_(self.modulator_conv.bias, 0.)
        
        self.regular_conv = nn.Conv2d(in_channels=in_channels,
                                      out_channels=out_channels,
                                      kernel_size=kernel_size,
                                      stride=stride,
                                      padding=self.padding,
                                      bias=bias)
    
    def forward(self, x):
        h, w = x.shape[2:]
        max_offset = max(h, w)/4.

        offset = self.offset_conv(x).clamp(-max_offset, max_offset)
        modulator = 2. * torch.sigmoid(self.modulator_conv(x))
        
        x = torchvision.ops.deform_conv2d(input=x, 
                                          offset=offset, 
                                          weight=self.regular_conv.weight, 
                                          bias=self.regular_conv.bias, 
                                          padding=self.padding,
                                          mask=modulator
                                          )
        return x

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


import torch
from torch import nn


class MNISTClassifier(nn.Module):
    def __init__(self,
                 deformable=False):

        super(MNISTClassifier, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=True)   
        conv = nn.Conv2d if deformable==False else DeformableConv2d
        self.conv4 = conv(32, 32, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv5 = conv(32, 32, kernel_size=3, stride=1, padding=1, bias=True)
        
        self.pool = nn.MaxPool2d(2)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(32, 10)
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x) # [14, 14]
        x = torch.relu(self.conv2(x))
        x = self.pool(x) # [7, 7]
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        x = self.gap(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        return x


def train(model, loss_function, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()


def test(model, loss_function, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    num_data = 0
    with torch.no_grad():
        for data, target in test_loader:
            org_data, target = data.to(device), target.to(device)

            for scale in np.arange(0.5, 1.6, 0.1): # [0.5, 0.6, ... ,1.2, 1.3, 1.4, 1.5]
                data = transforms.functional.affine(org_data, scale=scale, angle=0, translate=[0,0],shear=0)
                output = model(data)
                test_loss += loss_function(output, target).item()  # sum up batch mean loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
                num_data += len(data)

    test_loss /= num_data

    test_acc = 100. * correct / num_data
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, num_data,
        test_acc))
    return test_acc


def main(use_deformable_conv=False):
    # Training settings
    seed=1
    setup_seed(seed)

    use_cuda = torch.cuda.is_available()
    batch_size = 64
    lr=1e-3
    gamma=0.7
    epochs=14

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 4,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    train_transform = transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    
    dataset1 = datasets.MNIST('./data', train=True, download=True,
                       transform=train_transform)
    dataset2 = datasets.MNIST('./data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = MNISTClassifier(use_deformable_conv).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    loss_function = nn.CrossEntropyLoss()
    best_test_acc = 0.
    for epoch in range(1, epochs + 1):
        train(model, loss_function, device, train_loader, optimizer, epoch)
        best_test_acc = max(best_test_acc, test(model, loss_function, device, test_loader))
        scheduler.step()
    print("best top1 acc(%): ", f"{best_test_acc:.2f}")

In [ ]:
main(use_deformable_conv=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: Average loss: 0.0101, Accuracy: 88833/110000 (80.76%)


Test set: Average loss: 0.0138, Accuracy: 83529/110000 (75.94%)


Test set: Average loss: 0.0078, Accuracy: 93126/110000 (84.66%)


Test set: Average loss: 0.0065, Accuracy: 96733/110000 (87.94%)


Test set: Average loss: 0.0066, Accuracy: 96024/110000 (87.29%)


Test set: Average loss: 0.0060, Accuracy: 97076/110000 (88.25%)


Test set: Average loss: 0.0055, Accuracy: 98812/110000 (89.83%)


Test set: Average loss: 0.0056, Accuracy: 98281/110000 (89.35%)


Test set: Average loss: 0.0056, Accuracy: 98391/110000 (89.45%)


Test set: Average loss: 0.0058, Accuracy: 97774/110000 (88.89%)


Test set: Average loss: 0.0053, Accuracy: 99030/110000 (90.03%)


Test set: Average loss: 0.0056, Accuracy: 98597/110000 (89.63%)


Test set: Average loss: 0.0055, Accuracy: 98845/110000 (89.86%)


Test set: Average loss: 0.0055, Accuracy: 98733/110000 (89.76%)

best top1 acc(%):  90.03


In [ ]:
main(use_deformable_conv=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: Average loss: 0.0067, Accuracy: 94507/110000 (85.92%)


Test set: Average loss: 0.0048, Accuracy: 99084/110000 (90.08%)


Test set: Average loss: 0.0047, Accuracy: 99246/110000 (90.22%)


Test set: Average loss: 0.0040, Accuracy: 100938/110000 (91.76%)


Test set: Average loss: 0.0044, Accuracy: 100445/110000 (91.31%)


Test set: Average loss: 0.0043, Accuracy: 100963/110000 (91.78%)


Test set: Average loss: 0.0041, Accuracy: 100745/110000 (91.59%)


Test set: Average loss: 0.0039, Accuracy: 101710/110000 (92.46%)


Test set: Average loss: 0.0036, Accuracy: 102096/110000 (92.81%)


Test set: Average loss: 0.0036, Accuracy: 102070/110000 (92.79%)


Test set: Average loss: 0.0038, Accuracy: 101747/110000 (92.50%)


Test set: Average loss: 0.0035, Accuracy: 102195/110000 (92.90%)


Test set: Average loss: 0.0037, Accuracy: 101947/110000 (92.68%)


Test set: Average loss: 0.0037, Accuracy: 102001/110000 (92.73%)

best top1 acc(%):  92.90
